In [23]:
import tensorflow as tf
from PIL import Image
import numpy as np

import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from multiprocessing import Process
from wait4it import wait_for
from typing import List
from fastapi.responses import HTMLResponse


from tensorflow.keras.applications.imagenet_utils import decode_predictions

from PIL import Image
from io import BytesIO

MY_IP_ADDRESS = '10.0.1.7'
PORT = 8080
app = FastAPI()

def run():
    uvicorn.run(app, port=PORT, host=MY_IP_ADDRESS)

_api_process = None

def start_api():
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()

    _api_process = Process(target=run, daemon=True)
    _api_process.start()
    wait_for(host=MY_IP_ADDRESS,port=PORT)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]

In [24]:
# following from tests in 1.0-kpr notebook and the FastAPI tutorial here
#     https://towardsdatascience.com/image-classification-api-with-tensorflow-and-fastapi-fc85dc6d39e8
#
# also note issues w TF and predict w/in serving, here
# https://stackoverflow.com/questions/58919924/tensorflow-keras-model-served-by-a-flask-app-uwsgi-gets-stuck-in-model-predict
# and
# https://stackoverflow.com/questions/50906673/keras-model-is-stuck-at-prediction-level-when-running-the-server-with-gunicorn

def load_model():
    model = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=True, 
        input_shape=(224, 224, 3)
    )
    print("Model loaded")
    return model

model = load_model()

# https://github.com/keras-team/keras/issues/12379
graph = tf.keras.graph()

def predict(image: Image.Image):
    with graph.as_default():     #predict here    
        image = np.asarray(image.resize((224, 224)))[..., :3]
        image = np.expand_dims(image, 0)
        image = image / 127.5 - 1.0
        print('resized image... about to decode predictions...')
        result = decode_predictions(model.predict(image), 2)[0]
        print('... have result!')    
        response = []
        for i, res in enumerate(result):
            resp = {}
            resp["class"] = res[1]
            resp["confidence"] = f"{res[2]*100:0.2f} %"
            response.append(resp)
        return response

def read_imagefile(file) -> Image.Image:
    image = Image.open(BytesIO(file))
    return image

Model loaded


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
@app.post("/predict/image")
async def predict_api(files: List[UploadFile] = File(...)):
    file = files[0]
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    image = read_imagefile(await file.read())
    prediction = predict(image)
    print(prediction)
    return prediction

@app.get("/")
async def main():
    content = """
<body>
<form action="/predict/image" enctype="multipart/form-data" method="post" multiple>
<input name="files" type="file">
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)

In [21]:
start_api()

INFO:     Started server process [24156]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://10.0.1.7:8080 (Press CTRL+C to quit)


INFO:     10.0.1.36:61902 - "GET / HTTP/1.1" 200 OK
resized image... about to decode predictions...


2021-07-27 15:07:00.075870: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-27 15:07:00.096500: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2098860000 Hz


In [22]:
_api_process.terminate()
_api_process.kill()